# Passos do pipeline

  1. **Controle de qualidade de sequências** - Visualizar pelo FastQC
  2. **Processo de Trimagem**
  3. **Alinhamento/Mapeamento de sequências** 
  4. **Chamada de Variantes** 


--------

# Pipeline Automatizado

In [ ]:
%%bash
# Executar o bloco de códigos abaixo para instalar programas
sudo apt install fastqc samtools bwa bcftools #instalacao direta na máquina do fastqc, samtools, bwa, bcftools
python3 -m pip install multiqc #instalacao via pip do multiqc
python3 -m pip install --user --upgrade cutadapt #instalacao via pip do cutadapt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash

#!/bin/bash

##### Google Colab #####
# Pelo Google Colab, executar o comando abaixo

# Fazer cópia do repositório
git clone https://github.com/Varstation/bioinfotest.git

# Obter dados
cd bioinfotest/data
bash data_files_download.sh

cd ../..

#cp bioinfotest/data/*.fasta /content/drive/MyDrive/Bioinfo_selecao_2022/DESAFIO_05
mkdir -p /content/drive/MyDrive/Bioinfo_selecao_2022/DESAFIO_05/output
##########################

# configuração e arquivos
FASTQ_R1="bioinfotest/data/510-7-BRCA_S8_L001_R1_001.fastq.gz"
FASTQ_R2="bioinfotest/data/510-7-BRCA_S8_L001_R2_001.fastq.gz"
GENOMA_REF="bioinfotest/data/hg19.fasta"
DIR="/content/drive/MyDrive/Bioinfo_selecao_2022/DESAFIO_05"
DIR_OUTPUT="/content/drive/MyDrive/Bioinfo_selecao_2022/DESAFIO_05/output"

#fastqc
fastqc $FASTQ_R1 --extract -o $DIR
fastqc $FASTQ_R2 --extract -o $DIR

# cutadapt to filter sequences
# Este passo não foi realizado, uma vez que as sequências estão com boa qualidade após a visualização pelo FastQC.
#~/.local/bin/cutadapt -q 30 -l 100 -o $DIR/read_R1.trim.fastq.gz -p $DIR/read_R2.trim.fastq.gz $FASTQ_R1 $FASTQ_R2
cp $GENOMA_REF $DIR/genoma_ref.fa

#bwa
bwa index $DIR/genoma_ref.fa

#If cutadapt was used:
#bwa mem $DIR/genoma_ref.fa $DIR/read_R1.trim.fastq.gz $DIR/read_R2.trim.fastq.gz > $DIR_OUTPUT/alignment.sam

#If not:
bwa mem $DIR/genoma_ref.fa $FASTQ_R1 $FASTQ_R2 > $DIR_OUTPUT/alignment.sam

samtools stats $DIR_OUTPUT/alignment.sam | grep ^SN | cut -f 2- > stats_bwa.txt

#chamada de variantes
samtools view -bS $DIR_OUTPUT/alignment.sam > $DIR_OUTPUT/alignment.bam
samtools sort $DIR_OUTPUT/alignment.bam> $DIR_OUTPUT/alignment.sort.bam
samtools index $DIR_OUTPUT/alignment.sort.bam
bcftools mpileup -Ou -f $DIR_OUTPUT/genoma_ref.fa $DIR_OUTPUT/alignment.sort.bam | bcftools call -mv -Ob -o $DIR_OUTPUT/snps.bcf
bcftools view $DIR_OUTPUT/snps.bcf > $DIR_OUTPUT/snps.vcf
bcftools view -v snps $DIR_OUTPUT/snps.vcf | wc -l > $DIR_OUTPUT/counts_snp.txt

1 - Quantas sequências de DNA de paciente sequenciados temos nos arquivos de fastqs R1 e R2 respectivamente ?

In [ ]:
# Há 64276 sequências no R1 e 64276 no R2.

2 - Sobre o genoma humano hg19, quantos contigs tem o nosso genoma hg19 (hg19.fasta) aqui disponibilizado para este pipeline ?

In [ ]:
%%bash
grep -c ">" bioinfotest/data/hg19.fasta

93


In [ ]:
# Há 93 contigs no arquivo fasta do genoma hg19.

3 - Quantos alinhamentos há na região chr17:41197694-41197819 ? 

In [ ]:
# Há __ alinhamentos na região chr17:41197694-41197819

4 - Quantos alinhamentos não conseguiram ser mapeados (unmapped alignments ?) 

In [ ]:
# 2663 alinhamentos das sequências não foram mapeados.

5 - Realize o alinhamento das sequências FASTQ contra o genoma de referência hg19 aqui disponibilizado, e realize a chamada de variantes utilizando a região alvo BRCA.list (interesse apenas na região dos genes BRCA1 e BRCA2).  Realize a anotação de dados funcionais usando o SNPEFF.
 Com este arquivo em mãos , responda as seguintes perguntas ?

5.1- Quantas variantes existem com impacto funcional (Annotation_Impact) do tipo HIGH, MODERATE, LOW ? Existe alguma variante em HIGH ? Qual é cromossomo, posição e a alteração ?